#### Check De Data

In [6]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
import dvc.api
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

In [12]:
path="/data/data/prepared/beer_profile_and_ratings.csv"
repo="G:\My Drive\PreTraineeship"
version="V1"
remote="storage"

In [13]:
data_url = dvc.api.get_url(path=path)
print(data_url)